# 시계열 모델링_따릉이

* 지금까지 배운 것을 복습 합니다.
* Data : 서울 공유 자전거
* 문제 : 2시간 후의 수요를 예측하고자 합니다.
* 최소 2개 이상의 모델을 만들고 성능을 비교해 봅시다.
------

![](https://mediahub.seoul.go.kr/uploads/mediahub/2021/03/RaKqiqgRXyNmYVYymXQIvCjPHpncuhBX.png)

## 1.환경준비

### (1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, Flatten, SimpleRNN, LSTM, Flatten, Input
from keras.backend import clear_session
from keras.optimizers import Adam

### (2) 필요한 함수 생성

* 학습곡선 함수

In [ ]:
# 학습곡선 함수
def dl_history_plot(history):
    plt.figure(figsize=(10,6))
    plt.plot(history['loss'], label='train_err', marker = '.')
    plt.plot(history['val_loss'], label='val_err', marker = '.')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

* 데이터 2 --> 3차원 변환

In [ ]:
# 시계열 데이터 전처리 2차원 --> 3차원으로 변환
def temporalize(x, y, timesteps):
    nfeature = x.shape[1]
    output_x = []
    output_y = []
    for i in range(len(x) - timesteps + 1):
        t = []
        for j in range(timesteps):
            t.append(x[[(i + j)], :])
        output_x.append(t)
        output_y.append(y[i + timesteps - 1])
    return np.array(output_x).reshape(-1,timesteps, nfeature), np.array(output_y)

### (3) 데이터로딩

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/SeoulBikeData2.csv'
data = pd.read_csv(path)
data.drop(['Visibility','Solar'], axis = 1, inplace = True)
data['DateTime'] = pd.to_datetime(data['DateTime'], format='%Y-%m-%d %H:%M:%S')
data.head()

**변수설명**  

* DateTime : year-month-day hh:mi:ss
* Count : 시간대별 수요량
* Temperature : 온도(섭씨)
* Humidity : 습도(%)
* WindSpeed : 풍속(m/s)
* Rainfall - mm
* Snowfall - cm
* Seasons - Winter, Spring, Summer, Autumn
* Holiday - Holiday / No holiday
* FuncDay - Yes / No


In [ ]:
# 데이터 기간은 다음과 같습니다.
data.DateTime.min(), data.DateTime.max()

In [ ]:
# 14일 동안의 수요량을 살펴 봅시다.
temp = data[:24*14]
plt.figure(figsize = (20,8))
plt.plot('DateTime', 'Count', data = temp)
plt.grid()
plt.show()

## 2.데이터 준비

### (1) y 만들기
* 2시간 이후의 수요량을 예측해야 합니다.

In [ ]:
data['y'] = data['Count'].shift(-2)
data.head()

In [ ]:
# 2칸을 앞당겼기 때문에 하위 2행의 y값에 NaN이 표시되어 있습니다.
data.tail()

In [ ]:
# 하위 2행은 삭제합니다.
# 하위 2행 제외하고 다시 붓기
data = data[:-2]

### (2) Feature Engineering

In [ ]:
# 요일
data['WeekDay'] = data['DateTime'].dt.dayofweek

# 주말여부
data['WeekEnd'] = np.where(data['WeekDay']>4, 1,0)

# hour
data['Hour'] = data['DateTime'].dt.hour
data.head()

### (3) x, y 분리

In [ ]:
target = 'y'
x = data.drop(['DateTime', target], axis = 1)
y = data.loc[:,target]

### (4) 가변수화

In [ ]:
cat_cols = ['Seasons','Holiday','FuncDay', 'WeekDay','Hour']
x = pd.get_dummies(x, columns = cat_cols, drop_first = True)

### (5) 스케일링

* y 값이 클때는 y도 스케일링 필요

In [ ]:
# x 스케일링
scaler_x = MinMaxScaler()
x_s = scaler_x.fit_transform(x)

In [ ]:
# y는 1차원이므로 수동으로.
y_max, y_min = y.max(), y.min()
y_s = (y - y_min) / (y_max - y_min)

### (6) 3차원 구조 만들기

In [ ]:
timestep =     # ⬅️ 수정
x2, y2 = temporalize(x_s, y_s, timestep)
x2.shape, y2.shape

### (7) 데이터 분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x2, y2, test_size= 24*14, shuffle = False)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_val.shape, y_val.shape

## 3.모델링1



### (1) 입력 구조(shape)
* 분석 단위 : 2차원 ( timesteps, nfeatures)

### (2) 모델 구조 설계

In [ ]:
model1 =

### (3) 컴파일 및 학습

### (4) 예측 및 평가

In [ ]:
# 예측
pred1 = model1.predict(x_val)

# y를 원래대로 돌려 놓기
y_val_real = y_val * (y_max - y_min) + y_min
pred_real  = pred1 * (y_max - y_min) + y_min

mean_absolute_error(y_val_real, pred_real)

In [ ]:
plt.figure(figsize = (20,6))
plt.plot(y_val_real, label = 'actual')
plt.plot(pred_real, label = 'predicted')
plt.legend()
plt.grid()
plt.show()

## 4.모델링2



### (1) 입력 구조(shape)
* 분석 단위 : 2차원 ( timesteps, nfeatures)

### (2) 모델 구조 설계

### (3) 컴파일 및 학습

### (4) 예측 및 평가